## 1. Importação das Bibliotecas e Carregamento dos Dados

In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics.pairwise import euclidean_distances
import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 8)

In [56]:
consumo_df = pd.read_csv('../data/consumo.csv')
clima_df = pd.read_csv('../data/clima.csv')
clientes_df = pd.read_csv('../data/clientes.csv')

consumo_df['date'] = pd.to_datetime(consumo_df['date'])
clima_df['date'] = pd.to_datetime(clima_df['date'])

## 2. Análise Detalhada

In [57]:
# Análise dos valores faltantes de temperatura
# verificando se os registros faltantes tem relação com região
temp_faltante = clima_df[clima_df['temperature'].isnull()].copy()
print(f"registros faltantes: {len(temp_faltante)}")
print("Distribuição por região:")
print(temp_faltante['region'].value_counts())
print("")

# verificando se os registros faltantes tem relação com mês
temp_faltante['mes'] = temp_faltante['date'].dt.month
print(temp_faltante['mes'].value_counts().sort_index())
print("")

registros faltantes: 45
Distribuição por região:
region
Leste     13
Norte     11
Centro     8
Oeste      7
Sul        6
Name: count, dtype: int64

mes
1     6
2     6
3    11
4     6
5     7
6     9
Name: count, dtype: int64



In [58]:
# Análise dos clientes com região desconhecida
clientes_desconhecidos = clientes_df[clientes_df['region'] == 'Desconhecida']['client_id'].tolist()
print(f"Clientes com região desconhecida: {clientes_desconhecidos}")

# Analisando padrão de consumo desses clientes
consumo_desconhecidos = consumo_df[consumo_df['client_id'].isin(clientes_desconhecidos)]
print(f"Relevância dos clientes: {len(consumo_desconhecidos)}")
print(consumo_desconhecidos['consumption_kwh'].describe())

# Comparando com clientes de regiões conhecidas
consumo_conhecidos = consumo_df[~consumo_df['client_id'].isin(clientes_desconhecidos)]
print(consumo_conhecidos['consumption_kwh'].describe())

Clientes com região desconhecida: ['C0015', 'C0050', 'C0058', 'C0077', 'C0092']
Relevância dos clientes: 900
count    900.000000
mean      14.608800
std        3.126769
min        6.290000
25%       12.422500
50%       14.535000
75%       16.872500
max       22.860000
Name: consumption_kwh, dtype: float64
count    17100.000000
mean        14.822564
std          3.757646
min          2.640000
25%         12.050000
50%         14.910000
75%         17.670000
max         27.920000
Name: consumption_kwh, dtype: float64


## 3. Tratamento dos Valores Faltantes de Temperatura

In [59]:
clima_tratado = clima_df.copy()

# Estratégia 1: Imputação por média regional e temporal
# Para cada valor faltante, usarei a média da mesma região em datas próximas
def imputar_temperatura_regional(df):
    df_temp = df.copy()
    
    for idx in df_temp[df_temp['temperature'].isnull()].index:
        regiao = df_temp.loc[idx, 'region']
        data = df_temp.loc[idx, 'date']
        
        # Buscar valores de temperatura da mesma região em um período de 7 dias
        inicio = data - pd.Timedelta(days=7)
        fim = data + pd.Timedelta(days=7)
        
        valores_proximos = df_temp[
            (df_temp['region'] == regiao) & 
            (df_temp['date'] >= inicio) & 
            (df_temp['date'] <= fim) & 
            (df_temp['temperature'].notna())
        ]['temperature']
        
        if len(valores_proximos) > 0:
            df_temp.loc[idx, 'temperature'] = valores_proximos.mean()
        else:
            # Se não houver valores próximos, usar a média geral da região
            media_regiao = df_temp[df_temp['region'] == regiao]['temperature'].mean()
            df_temp.loc[idx, 'temperature'] = media_regiao
    
    return df_temp

clima_imputado_1 = imputar_temperatura_regional(clima_tratado)
print(f"{clima_imputado_1['temperature'].isnull().sum()}")

0


In [60]:
# Estratégia 2: Imputação usando KNN considerando região, data e umidade
clima_knn = clima_df.copy()

# Codificando região
le_region = LabelEncoder()
clima_knn['region_encoded'] = le_region.fit_transform(clima_knn['region'])

# Criando features temporais
clima_knn['dia_do_ano'] = clima_knn['date'].dt.dayofyear
clima_knn['mes'] = clima_knn['date'].dt.month

# Selecionando features para KNN
features_knn = ['region_encoded', 'dia_do_ano', 'mes', 'humidity', 'temperature']
dados_knn = clima_knn[features_knn].copy()

# Aplicando KNN Imputer
knn_imputer = KNNImputer(n_neighbors=5)
dados_imputados = knn_imputer.fit_transform(dados_knn)

# Criando dataframe com dados imputados
clima_imputado_2 = clima_df.copy()
clima_imputado_2['temperature'] = dados_imputados[:, -1]

print(f"{clima_imputado_2['temperature'].isnull().sum()}")

# Comparando as duas estratégias
print("Comparação:")
print(f"Temperatura média original: {clima_df['temperature'].mean():.2f}°C")
print(f"Temperatura média após imputação regional: {clima_imputado_1['temperature'].mean():.2f}°C")
print(f"Temperatura média após KNN: {clima_imputado_2['temperature'].mean():.2f}°C")

clima_final = clima_imputado_1.copy()

0
Comparação:
Temperatura média original: 25.10°C
Temperatura média após imputação regional: 25.10°C
Temperatura média após KNN: 25.10°C


## 4. Tratamento dos Clientes com Região Desconhecida

In [61]:
# Estratégia: Imputar região baseada em similaridade de padrão de consumo

# Calculando perfil de consumo por cliente
perfil_consumo = consumo_df.groupby('client_id')['consumption_kwh'].agg([
    'mean', 'std', 'min', 'max', 'median'
]).reset_index()

# Adicionando região aos perfis
perfil_consumo = perfil_consumo.merge(clientes_df, on='client_id', how='left')

# Separando clientes conhecidos e desconhecidos
perfil_conhecidos = perfil_consumo[perfil_consumo['region'] != 'Desconhecida'].copy()
perfil_desconhecidos = perfil_consumo[perfil_consumo['region'] == 'Desconhecida'].copy()

# Calculando perfil médio por região conhecida
perfil_por_regiao = perfil_conhecidos.groupby('region')[['mean', 'std', 'min', 'max', 'median']].mean()
print(perfil_por_regiao.round(2))

         mean   std   min    max  median
region                                  
Centro  14.95  2.24  8.98  20.66   14.95
Leste   14.97  2.25  8.99  20.83   14.99
Norte   14.25  2.26  8.08  20.13   14.25
Oeste   15.83  2.26  9.82  22.32   15.77
Sul     14.69  2.26  8.67  20.84   14.70


In [62]:
def encontrar_regiao_similar(cliente_perfil, perfis_regionais):

    features = ['mean', 'std', 'min', 'max', 'median']
    
    # Preparando os dados para normalização
    dados_completos = pd.concat([
        perfis_regionais[features],
        cliente_perfil[features].to_frame().T
    ], ignore_index=True)
    
    # Normalizando os dados usando MinMaxScaler
    scaler = MinMaxScaler()
    dados_normalizados = scaler.fit_transform(dados_completos)
    
    # Separando dados normalizados
    perfis_norm = dados_normalizados[:-1]
    cliente_norm = dados_normalizados[-1:]
    
    # Calculando distâncias
    distancias_array = euclidean_distances(cliente_norm, perfis_norm)[0]
    
    # Criando dicionário de distâncias por região
    distancias = dict(zip(perfis_regionais.index, distancias_array))
    
    # Retornando região com menor distância
    regiao_similar = min(distancias, key=distancias.get)
    return regiao_similar, distancias

# Imputando região para clientes desconhecidos
clientes_tratado = clientes_df.copy()
imputacoes_realizadas = []

for idx, cliente in perfil_desconhecidos.iterrows():
    cliente_id = cliente['client_id']
    regiao_similar, distancias = encontrar_regiao_similar(cliente, perfil_por_regiao)
    
    # Atualizando a região
    clientes_tratado.loc[clientes_tratado['client_id'] == cliente_id, 'region'] = regiao_similar
    
    imputacoes_realizadas.append({
        'client_id': cliente_id,
        'regiao_imputada': regiao_similar,
        'consumo_medio': cliente['mean'],
        'distancias': distancias
    })

for imp in imputacoes_realizadas:
    print(f"Cliente {imp['client_id']}: {imp['regiao_imputada']}")

Cliente C0015: Oeste
Cliente C0050: Centro
Cliente C0058: Norte
Cliente C0077: Oeste
Cliente C0092: Norte


## 5. Salvamento dos Dados Tratados

In [63]:
clima_final.to_csv('../data/clima_tratado.csv', index=False)
clientes_tratado.to_csv('../data/clientes_tratado.csv', index=False)
consumo_df.to_csv('../data/consumo_tratado.csv', index=False)